In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
# Path to the original train.json
original_json_path = "/content/drive/MyDrive/fathomnet-out-of-sample-detection/resampled/top50-train.json"

# Load the JSON file
with open(original_json_path, "r") as f:
    data = json.load(f)

# Renumber `id` in annotations
for idx, ann in enumerate(data["annotations"], start=1):
    ann["id"] = idx  # Start numbering from 1

# Save the updated JSON file
updated_json_path = "/content/drive/MyDrive/fathomnet-out-of-sample-detection/resampled/top50-train-renumbered.json"
with open(updated_json_path, "w") as f:
    json.dump(data, f)

NameError: name 'json' is not defined

In [ ]:
import os
import json

# Paths to image directories
train_images_dir = "/content/drive/MyDrive/fathomnet-out-of-sample-detection/resampled/images/train"
val_images_dir = "/content/drive/MyDrive/fathomnet-out-of-sample-detection/resampled/images/val"

# Path to the original train.json
original_json_path = "/content/drive/MyDrive/fathomnet-out-of-sample-detection/resampled/top50-train-renumbered.json"

# Load the JSON file
with open(original_json_path, "r") as f:
    data = json.load(f)

# Get image file names from each folder
train_image_files = set(os.listdir(train_images_dir))
val_image_files = set(os.listdir(val_images_dir))

img_names = [f for f in os.listdir(train_images_dir) if f.lower().endswith(".png")]
print(data['annotations'])
train_images = [img for img in data['images'] if img['file_name'] in img_names]
train_annotations = [ann for ann in data['annotations'] if ann['image_id'] in [img['id'] for img in train_images]]
val_images = [img for img in data['images'] if img['file_name'] not in img_names]
val_annotations = [ann for ann in data['annotations'] if ann['image_id'] in [img['id'] for img in val_images]]
print(val_annotations)

# # Filter images and annotations for training
# train_images = [img for img in data["images"] if img["file_name"] in train_image_files
# # Filter images and annotations for training
# train_images = [img for img in data["images"] if img["file_name"] in train_image_files]
# train_image_ids = {img["id"] for img in train_images}
# train_annotations = [ann for ann in data["annotations"] if ann["image_id"] in train_image_ids]

# # Filter images and annotations for validation
# val_images = [img for img in data["images"] if img["file_name"] in val_image_files]
# val_image_ids = {img["id"] for img in val_images}
# val_annotations = [ann for ann in data["annotations"] if ann["image_id"] in val_image_ids]

# Create new JSON data for train and val
train_data = {
    "info": data["info"],
    "images": train_images,
    "licenses": data["licenses"],
    "annotations": train_annotations,
    "categories": data["categories"],
}

val_data = {
    "info": data["info"],
    "images": val_images,
    "licenses": data["licenses"],
    "annotations": val_annotations,
    "categories": data["categories"],
}
json_data_train = json.dumps(train_data, indent=4)
json_data_val = json.dumps(val_data, indent=4)

# # Save the new JSON files
with open("/content/drive/MyDrive/fathomnet-out-of-sample-detection/resampled/train_split.json", "w") as f:
    f.write(json_data_train)

with open("/content/drive/MyDrive/fathomnet-out-of-sample-detection/resampled/val_split.json", "w") as f:
    f.write(json_data_val)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print(f"Training images: {len(train_images)}")
print(f"Training annotations: {len(train_annotations)}")
print(f"Validation images: {len(val_images)}")
print(f"Validation annotations: {len(val_annotations)}")


Training images: 4688
Training annotations: 19625
Validation images: 1172
Validation annotations: 3632


In [ ]:
from torchvision.transforms import functional as F
import torch

class CocoTransform:
    def __call__(self, image, target):
        # Convert the image to a tensor
        image = F.to_tensor(image)

        # Filter valid boxes
        valid_boxes = []
        valid_labels = []
        for obj in target:
            x, y, w, h = obj["bbox"]
            if w > 0 and h > 0:  # Check if width and height are positive
                valid_boxes.append([x, y, x + w, y + h])  # Convert to [x_min, y_min, x_max, y_max]
                valid_labels.append(obj["category_id"])

        # Convert to PyTorch tensors
        target = {
            "boxes": torch.as_tensor(valid_boxes, dtype=torch.float32),
            "labels": torch.as_tensor(valid_labels, dtype=torch.int64),
        }
        return image, target


In [ ]:
print(train_data['annotations'])

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
from torchvision.datasets import CocoDetection

train_dataset = CocoDetection(
    root=train_images_dir,
    annFile="/content/drive/MyDrive/fathomnet-out-of-sample-detection/resampled/train_split.json",
    transforms=CocoTransform()
)

val_dataset = CocoDetection(
    root=val_images_dir,
    annFile="/content/drive/MyDrive/fathomnet-out-of-sample-detection/resampled/val_split.json",
    transforms=CocoTransform()
)


loading annotations into memory...
Done (t=0.22s)
creating index...
index created!
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [ ]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    images, targets = zip(*batch)
    return list(images), list(targets)

train_loader = DataLoader(
    train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn
)

val_loader = DataLoader(
    val_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn
)


In [ ]:
import torch
from torch.optim import SGD
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from pycocotools.cocoeval import COCOeval
import numpy as np

# Number of classes (including background)
num_classes = 290 + 1

# Initialize Faster R-CNN
model = fasterrcnn_resnet50_fpn(weights="DEFAULT")
in_features = model.roi_heads.box_predictor.cls_score.in_features
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Optimizer
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0005)

# Device setup
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:01<00:00, 157MB/s]


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

def display_predictions(images, outputs, class_names, threshold=0.5):
    """
    Display images with bounding box predictions.

    Args:
    - images: List of images (tensors).
    - outputs: Model outputs (list of dictionaries with 'boxes', 'labels', 'scores').
    - class_names: List of class names for labels.
    - threshold: Confidence threshold for displaying predictions.
    """
    for img, output in zip(images, outputs):
        # Convert image tensor to NumPy array
        img_np = img.permute(1, 2, 0).cpu().numpy()

        # Create a plot
        fig, ax = plt.subplots(1, figsize=(12, 8))
        ax.imshow(img_np)
        ax.axis("off")

        # Plot bounding boxes and labels
        for box, label, score in zip(output['boxes'], output['labels'], output['scores']):
            if score < threshold:
                continue

            # Unpack box coordinates
            x_min, y_min, x_max, y_max = box.cpu().numpy()
            width, height = x_max - x_min, y_max - y_min

            # Add a rectangle for the bounding box
            rect = patches.Rectangle(
                (x_min, y_min),
                width,
                height,
                linewidth=2,
                edgecolor='red',
                facecolor='none'
            )
            ax.add_patch(rect)

            # Add label and confidence
            label_text = f"{class_names[int(label)]}: {score:.2f}"
            ax.text(
                x_min, y_min - 10, label_text,
                color='red', fontsize=12, backgroundcolor='white'
            )

        plt.show()


In [ ]:
for images, raw_targets in train_loader:
    print("Raw targets:", raw_targets)
    break

Raw targets: [{'boxes': tensor([[1241.,  169., 1429.,  458.]]), 'labels': tensor([85])}, {'boxes': tensor([[458.,  72., 507., 108.],
        [ 98., 270., 153., 305.],
        [441., 148., 495., 185.],
        [277., 113., 330., 151.],
        [137., 156., 197., 193.]]), 'labels': tensor([160, 160, 160, 160, 160])}, {'boxes': tensor([[ 955.,  743., 1282.,  980.]]), 'labels': tensor([51])}, {'boxes': tensor([[296., 261., 357., 314.],
        [406., 193., 463., 236.]]), 'labels': tensor([160, 160])}, {'boxes': tensor([[181., 255., 236., 311.],
        [476., 246., 498., 272.],
        [300., 176., 324., 201.]]), 'labels': tensor([120, 160, 160])}, {'boxes': tensor([[ 565.,  204., 1622.,  815.],
        [   2.,   62.,  431.,  190.]]), 'labels': tensor([242, 219])}, {'boxes': tensor([[1300.,  809., 1465.,  948.],
        [1568.,  693., 1678.,  778.]]), 'labels': tensor([  9, 116])}, {'boxes': tensor([[553., 240., 621., 296.],
        [338., 214., 399., 259.]]), 'labels': tensor([160, 160])}

In [ ]:
from tqdm import tqdm
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import torch

# Number of epochs
num_epochs = 200

# Metrics storage
train_losses = []
val_map_50 = []
val_map_50_95 = []

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch + 1}/{num_epochs}")

    # Training Loop
    model.train()
    total_loss = 0
    train_progress = tqdm(train_loader, desc="Training", leave=False)

    for images, raw_targets in train_progress:
        # Move data to GPU
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in raw_targets]

        # Skip empty targets (no valid bounding boxes)
        valid_indices = [i for i, t in enumerate(targets) if len(t["boxes"]) > 0]
        images = [images[i] for i in valid_indices]
        targets = [targets[i] for i in valid_indices]

        if len(images) == 0:  # Skip batch if no valid targets
            continue

        # Forward pass and optimization
        optimizer.zero_grad()
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        optimizer.step()

        total_loss += losses.item()
        train_progress.set_postfix({"Loss": total_loss / len(train_loader)})

    avg_train_loss = total_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    print(f"Training Loss: {avg_train_loss:.4f}")

    # Validation Loop
    model.eval()
    predictions = []
    ground_truths = []

    val_progress = tqdm(val_loader, desc="Validating", leave=False)
    with torch.no_grad():
        for images, raw_targets in val_progress:
            # Move data to GPU
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in raw_targets]

            # Extract image IDs
            image_ids = [t.get("image_id", i) for i, t in enumerate(raw_targets)]

            # Skip empty targets
            valid_indices = [i for i, t in enumerate(targets) if len(t["boxes"]) > 0]
            images = [images[i] for i in valid_indices]
            targets = [targets[i] for i in valid_indices]
            image_ids = [image_ids[i] for i in valid_indices]

            if len(images) == 0:  # Skip batch if no valid targets
                continue

            # Forward pass
            outputs = model(images)

            # Collect predictions in COCO format
            for img_id, output, target in zip(image_ids, outputs, targets):
                boxes = output["boxes"].cpu().numpy()
                scores = output["scores"].cpu().numpy()
                labels = output["labels"].cpu().numpy()
                for box, score, label in zip(boxes, scores, labels):
                    x_min, y_min, x_max, y_max = box
                    predictions.append({
                        "image_id": img_id,
                        "category_id": int(label),
                        "bbox": [x_min, y_min, x_max - x_min, y_max - y_min],
                        "score": float(score),
                    })

                # Collect ground truths in COCO format
                for idx, (box, label) in enumerate(zip(target["boxes"], target["labels"])):
                    x_min, y_min, x_max, y_max = box.tolist()
                    ground_truths.append({
                        "id": len(ground_truths) + idx,  # Generate unique ID for each annotation
                        "image_id": img_id,
                        "category_id": int(label.item()),
                        "bbox": [x_min, y_min, x_max - x_min, y_max - y_min],
                        "area": (x_max - x_min) * (y_max - y_min),
                        "iscrowd": 0,
                    })

    # COCO Evaluation
    coco_gt = COCO()
    coco_gt.dataset = {
        "images": [{"id": gt["image_id"]} for gt in ground_truths],
        "annotations": ground_truths,
        "categories": [{"id": i, "name": str(i)} for i in range(num_classes)],
    }
    coco_gt.createIndex()

    coco_dt = coco_gt.loadRes(predictions)
    coco_eval = COCOeval(coco_gt, coco_dt, "bbox")
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    # Retrieve validation metrics
    map_50 = coco_eval.stats[1]
    map_50_95 = coco_eval.stats[0]
    val_map_50.append(map_50)
    val_map_50_95.append(map_50_95)
    print(f"Validation Results - mAP@50: {map_50:.4f}, mAP@50-95: {map_50_95:.4f}")



Epoch 1/200


Training Loss: 0.6855


creating index...
index created!
Loading and preparing results...
DONE (t=0.18s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=4.85s).
Accumulating evaluation results...
DONE (t=0.13s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.008
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.013
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.009
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.004
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.011
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.007
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.002
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.012
 Average Recall     (AR) @[ IoU=0.5

Training Loss: 0.4726


creating index...
index created!
Loading and preparing results...
DONE (t=0.03s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=5.22s).
Accumulating evaluation results...
DONE (t=0.13s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.011
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.022
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.010
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.018
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.013
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.012
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.004
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.012
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.026
 Average Recall     (AR) @[ IoU=0.5

...

Training Loss: 0.0650


creating index...
index created!
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=4.85s).
Accumulating evaluation results...
DONE (t=0.15s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.068
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.129
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.059
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.068
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.058
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.084
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.077
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.117
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.127
 Average Recall     (AR) @[ IoU=0.5

Training Loss: 0.1022


creating index...
index created!
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=4.83s).
Accumulating evaluation results...
DONE (t=0.16s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.066
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.139
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.053
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.074
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.048
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.099
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.067
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.139
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.151
 Average Recall     (AR) @[ IoU=0.5

Training:  48%|████▊     | 142/293 [02:43<02:50,  1.13s/it, Loss=0.0355]

In [ ]:
import matplotlib.pyplot as plt

# Plot Training Loss
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), train_losses, label="Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss Over Epochs")
plt.legend()
plt.savefig("training_loss.png")  # Save the plot
plt.show()

# Plot Validation mAP
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), val_map_50, label="mAP@50")
plt.plot(range(1, num_epochs + 1), val_map_50_95, label="mAP@50-95")
plt.xlabel("Epoch")
plt.ylabel("mAP")
plt.title("Validation mAP Over Epochs")
plt.legend()
plt.savefig("validation_map.png")  # Save the plot
plt.show()
